In [17]:
import cv2
import numpy as np
from ultralytics import YOLO 
import serial
from keras.models import load_model

In [18]:
# Load your VGG16 waste classification model
model_classification = load_model('vgg16_biodegradable_classifier_model.h5')

In [19]:
# Initialize YOLO model
model_yolo = YOLO('yolov8n.pt')
# object classes
classNames = model_yolo.names

In [22]:
# Open serial connection to Arduino
ser = serial.Serial('COM3', 9600)  # Replace 'COMX' with your Arduino's COM port

In [23]:
# Function for waste classification
def classify_waste(object_image):
    # Resize the object image to match your VGG16 input size
    resized_img = cv2.resize(object_image, (60, 60))
    # Expand dimensions to make it compatible with the model
    resized_img = np.expand_dims(resized_img, axis=0)
    # Perform classification
    prediction = model_classification.predict(resized_img)
    # Convert prediction to label
    label = np.argmax(prediction)
    # Map label to waste type
    if label == 0:
        return "biodegradable"
    else:
        return "non-biodegradable"

In [21]:
# start webcam

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    success, img = cap.read()
    results = model_yolo(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            cropped_object = img[int(y1):int(y2), int(x1):int(x2)]
            waste_type = classify_waste(cropped_object)

            # Send waste type to Arduino
            ser.write(waste_type.encode())  
            
            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # Display waste type on the frame
            cv2.putText(img, waste_type, (int(x1), int(y1) - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 3 persons, 314.8ms
1/1 [==============================] - 0s 83ms/step
Speed: 3.5ms preprocess, 314.8ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 248.0ms
1/1 [==============================] - 0s 100ms/step
Speed: 8.1ms preprocess, 248.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 203.5ms
1/1 [==============================] - 0s 97ms/step
Speed: 5.7ms preprocess, 203.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 192.8ms
1/1 [==============================] - 0s 74ms/step
Speed: 1.5ms preprocess, 192.8ms inference, 3.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 192.4ms
1/1 [==============================] - 0s 103ms/step
Speed: 3.3ms preprocess, 192.4ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 270.2ms
1/1 [==============================] - 0s 97ms/step
Speed: 3.1ms p